# Минимизация функций - как компьютер учится

В предыдущих блокнотах мы видели, что, изменяя **параметры** в функции, мы могли найти «наилучшее соответствие» этой функции некоторым данным. Мы используем **функцию потери**, чтобы количественно оценить «добротность» набора параметров, и мы ищем те параметры, которые **оптимизируют**, фактически **минимизируют**, функцию потери. 

Если мы учим машину, как самостоятельно минимизировать функцию потерь, мы говорим, что машина способна **научиться** моделировать наши данные. В последнем блокноте мы добавили такие термины, как **производная**, **градиент** и **градиентный спуск**, чтобы получить общее представление о том, как минимизировать функцию на компьютере. В этом блокноте мы рассмотрим эти концепции более тщательно, чтобы иметь возможность реализовать их с помощью Julia.

## Минимизация 1D-функции с использованием исчисления

Давайте снова нарисуем 1D функцию потерь $ L_1 $ из предыдущего ноутбука:

In [ ]:
σ(x) = 1 / (1 + exp(-x))
f(x, w) = σ(w * x)

x1 = 2
y1 = 0.8

L1(w) = (y1 - f(x1, w))^2

In [ ]:
using Plots; gr()
using Interact

In [ ]:
plot(L1, -2, 1.5, xlabel="w", ylabel="L1(w)", leg=false)

На глаз мы видим, что минимум составляет около $ w = 0.6 $. Но как мы можем заставить компьютер работать самостоятельно?

В предыдущей записной книжке мы рассматривали этот функциональный график как холм, если смотреть со стороны. Мы могли бы найти минимум, сделав холм липким и позволив мячу скатиться по нему. Мяч найдет и установит минимум функции. 

Теперь давайте посмотрим, как научить компьютер делать это. Нам нужно найти направление спуска вдоль холма, которое связано с его *уклоном* (насколько он крут). Математика предоставляет нам инструменты для расчета этого наклона!

А именно, наклон кривой $ L_1 (w) $ при $ w $ определяется ее **производной** $ L_1 '(w) $; геометрически это наклон **касательной линии** к кривой в этой точке, то есть прямая линия, которая касается кривой в этой точке.

Исчисление предоставляет нам некоторые правила для расчета аналитической формулы для производной, и мы увидим позже, как применять эти правила, хотя и косвенно, для машинного обучения. Однако, давайте попробуем заставить компьютер помогать нам находить производные численно!

## Аппроксимация производных

Напомним, что производная $ L_1 '(w) $ функции определяется как

$$L_1'(w) \simeq \frac{L_1(w+h) - L_1(w)}{h},$$

для маленького шага размером $ h $. (Строго говоря, мы должны взять предел, когда $ h $ стремится к $ 0 $, чтобы получить точное значение производной.)

#### Упражнение 1

Напишите функцию для вычисления производной функции в данной точке. Обратите внимание, что в Julia мы можем легко передавать функции в качестве аргументов другим функциям! Функция должна принимать функцию, точку $ w $, в которой нужно вычислить производную, и значение $ h $, которое по умолчанию должно иметь значение 0.001. 

*Примечание*: входной аргумент функции может иметь *значение по умолчанию*, если мы устанавливаем входной аргумент равным этому значению по умолчанию при определении функции. Например,

```julia
f(x, a = 3) = a * x^2
```

Функция `f` возведет в квадрат введенное нами значение и умножит на` a`. Однако, если мы решим вызывать `f(x)` не передавая ему входное значение `a`, оно будет считать, что` a` равно `3`, и возвращать` 3 * x ^ 2`.

#### Упражнение 2

Запишите интерактивную визуализацию касательной к графику $ L_1 $, чтобы мы могли визуализировать касательную в любой точке $ L_1 $. Включите текущее значение производной в заголовке.

*Подсказка*: Напомним, что прямая линия через точку $ (x_0, y_0) $ с наклоном $ m $ определяется как

$$ \frac{y - y_0}{x - x_0} = m,$$

следовательно 

$$y = y_0 + m*(x - x_0).$$

#### Упражнение 3

Каково значение производной (наклон касательной линии) в минимуме? Это может случиться где-нибудь еще?

## Минимизация градиентным спуском

[Доп теория и реализация](https://habr.com/ru/post/440070/)

Касательная в точке является хорошим приближением функции вблизи этой точки. Следовательно, производная говорит нам, в каком направлении и как быстро функция увеличивается или уменьшается, когда мы удаляемся на небольшое расстояние от этой точки. 

Как мы видели в предыдущей записной книжке, мы можем думать о функции как о холме и о том, что у нас есть шар, который мы хотим сдвинуть с холма. Гравитация автоматически вытянет мяч в направлении *вниз* в гору; мы хотим подражать этому в компьютере!

#### Упражнение 4

Если производная $ L_1 '(w) $ положительна, это означает, что $ L_1 $ увеличивается слева направо в точке $ w $. 

Если производная $ L_1 '(w) $ положительна (&gt; 0), в каком направлении мы должны двигаться $ w $, чтобы *уменьшить* $ L_1 $?

#### Упражнение 5

Если производная $ L_1 '(w) $ отрицательна (<0), то в каком направлении нам следует перемещать $ w $, чтобы *уменьшить* $ L_1 $?

Мы можем использовать эту информацию, чтобы сообщить компьютеру, каким способом сделать шаг. Это составляет численный алгоритм, называемый **градиентный спуск**; это называется так, поскольку мы спускаемся (движемся вниз) вдоль графика функции, используя информацию о ее градиенте (наклоне).

#### Упражнение 6

Реализуйте градиентный спуск, следуя этому рецепту для **итеративного (повторяющегося) алгоритма**:

1. Начните с первоначального предположения $ w_0 $ для $ w $.

2. На каждом шаге вычисляйте производную $ L_1 '(w_n) $ по текущему значению $ w_n $, используя функцию, которую вы создали выше. 

3. Измените $ w $ небольшим кратным (например, $ \eta = 0.01 $) значением только что созданной производной, с помощью

$w_{n+1} = w_n - \eta L_1'(w_n)$.

Для этой проблемы начните с $ w_0 = -2.0 $. Повторите шаги 2 и 3 «2000» раз. 

Упакуйте этот код в функцию, называемую `gradientDescent`, которая принимает два входа, функцию и диапазон значений $ w $ и возвращает окончательное значение $ w $ и $ L_1 (w) $. 

Используя `L1` и` -2: 0.01: 1.5` в качестве входных данных для своей функции, выясните для какого значения $ w $ достигается минимум для $ L_1 $?

#### Упражнение 7

Измените свой код для градиентного спуска, чтобы он возвращал результат, после того как он нашел ответ в пределах некоторой *погрешности*, вместо того, чтобы предпринимать заданное количество шагов. Новый рецепт для этого алгоритма:

1. Начните с первоначального предположения $ w_0 $ для $ w $.

2. На каждом шаге вычисляйте производную $ L_1 '(w_n) $ по текущему значению $ w_n $, используя функцию, которую вы создали выше. 

3. Измените $ w $ небольшим кратным (например, $ \eta = 0.01 $) значением только что созданной производной, с помощью

$w_{n+1} = w_n - \eta L_1'(w_n)$.

4. Проверьте, насколько $ w_ {n + 1} $ отличается от $ w_n $. Если вы удовлетворены тем, что $ L_1 (w_ {n + 1}) $ минимизировано, верните $ w_ {n + 1} $ и $ L_1 (w_ {n + 1}) $. В противном случае перейдите к шагу (2) и продолжите.

Отредактируйте `Gradient_descent` так, чтобы он принимал три входа: функцию, диапазон значений $ w $ и допустимую погрешность, указывающую, насколько близко $ w_ {n + 1} $ должна быть к $ w_n $, прежде чем вы сможете прекратить итерацию.

Используя `L1`,` -2: 0.01: 1.5` и `.000001` в качестве входных данных для градиента, найдите $ w $ при котором достигается минимум $ L_1 $?

#### Упражнение 8

Измените функцию `Gradient_descent`, чтобы она сохраняла результаты` (w, L1 (w)) `на каждом шаге алгоритма в виде массива и возвращала этот массив. Сколько шагов алгоритм выполняет для входных параметров `(L1, -2: 0.01: 1.5, .000001)` перед завершением? Вы должны посчитать начальный $ w_0 $ в качестве первого шага.

#### Упражнение 9

Наложите шаги, сделанные в последнем упражнении, на график $ L_1 (w) $ против $ w $. 

Где наш алгоритм делает самые большие шаги? (Где мяч движется быстрее всего вниз по склону?)

A) Между w = -2:-1<br>
B) Между w = -1:0<br>
C) Между w = 0:.6

*Под сказка*: Может быть легче увидеть, что происходит, если вы строите график, например, каждый 15-й шаг.

B) Следующий код генерирует желаемую фигуру:

```julia
wsteps = gradient_descent(C1, -2:0.01:1.5, .000001)
fig1 = plot(C1, -2, 1.5, xlabel="w", ylabel="C1(w)", leg=false)
for step in wsteps[1:15:end]
   scatter!([step[1]], [step[2]]) 
end
display(fig1)
```

## Функции двух переменных и их производные

До сих пор мы рассматривали функцию минимизации потерь $ L_1 (w) $, которая зависит от одного параметра, $ w $. Теперь давайте обратимся к функции ошибок $ L_2 (w, b) $ из предыдущего блокнота, которая является функцией *двух* параметров, $ w $ и $ b $, и попробуем минимизировать ее. 

Как мы уже видели, получается **поверхность** вместо кривой, когда мы строим график $ L_2 $ как функцию обоих его параметров.

**Упражнение 10** 

Нарисуйте график поверхности $ L_2 $, заданный
$$L_{2}(w, b) = \sum_i(y_i - g(x_i, w, b))^2$$
используя функцию `surface` из` Plots.jl`. Для этого графика используйте значения `xs` и` ys` из блокнота 5:
```julia
xs = [2, -3, -1, 1]
ys = [0.8, 0.3, 0.4, 0.4]
```
Мы можем получить хороший интерактивный 3D-график, используя бэкэнд Plotly `Plots.jl`, выполнив команду
```
plotly()
```

### Нахождение минимума

Вращая график, мы можем увидеть, что $ L_2 $ имеет единственный минимум. Мы хотим найти значения $ w $ и $ b $, где достигается это минимальное значение.

Следуя тому, что мы сделали для функции $ L_1 $ выше, мы ожидаем, что нам нужно будет вычислить производные от $ L_2 $. Так как функция более сложная, с производными придется повозиться!

Оказывается, правильная концепция [**градиента**](https://ru.wikipedia.org/wiki/Градиент) $ L_2 $, обозначается $ \nabla L_2 (w, b) $. Это **вектор**, состоящий из $ 2 $ чисел, если есть $ 2 $ параметра [или вообще $ n $ чисел, если есть $ n $ параметров].

Числа, образующие градиент $ \nabla L_2 (w, b) $, называются **частными производными** от $ L_2 $ по $ w $ и $ b $, записанными как
$$\frac{\partial L_2}{\partial w} \quad \text{and} \quad \frac{\partial L_2}{\partial b}.$$

Хотя это обозначение может показаться сложным, все это означает, что мы вычисляем производные так же, как и раньше, за исключением того, что мы фиксируем значение другой переменной.   Например, чтобы вычислить частную производную $ L_2 $ по отношению к $ w $, $ \frac {\partial L_2} {\partial w} $, мы фиксируем значение $ b $ и рассматриваем полученную функцию как функция единственной переменной $ w $; тогда мы используем формулу для производных функций от одной переменной.

[Обратите внимание, что $ \frac {\partial L_2} {\partial w} $ сама является функцией $ w $ и $ b $; мы могли бы написать $ \frac {\partial L_2} {\partial w} (w, b) $.]

#### Упражнение 11

Напишите функции, которые позволят вам вычислить частные производные функции $ f $ от двух переменных. 

В частности, объявите функции с именами `part_w` и `part_b`. Каждый из них должен иметь четыре входа - функцию $ f $ с двумя переменными, первый входной аргумент для $ f $, второй входной аргумент для $ f $ и размер шага `h` со значением по умолчанию` 0.001`. `part_w` должен возвращать частную производную $ f $ по первому входному аргументу, а `part_b` должен возвращать частную производную $ f $ по второму входному аргументу.

#### Упражнение 12

Используйте `partial_b` из последнего упражнения, чтобы найти частную производную $ L_2 $ по $ w $ при b = 0,3, $ \frac {\partial L_2} {\partial w} | _ {b = 0,3} $ для `w = -2:0.01:1`

#### Упражнение 13

Постройте сечение поверхности $ L_2 (w, b) $ при $ b = 0,3 $. Сделайте этот график интерактивным с `@manipulate`, чтобы показать, что функция `partial_w` задает наклон касательной к этому сечению для любой точки` w` в диапазоне `-2: 0,01: 1`. Для какого значения $ w $ в этом диапазоне наклон поперечного сечения ближе всего к -1?

## ***Необязательно***: функции с $ n $ входами 

Если функция $ f $ принимает $ n $ входных аргументов, мы можем записать их как $ p_1, \ldots, p_n $, где $ p_i $ означает «$ i $ -й параметр». В Julia мы можем заключить их в один **вектор**. Теперь мы можем вычислить частную производную $ \frac {\partial L_2} {\partial p_i} $ по переменной $ p_i $.

#### Упражнение 14

Для следующего упражнения вам нужно будет использовать команду splat `...`. Вы можете использовать эту команду, чтобы «открыть» коллекцию и передать все элементы этой коллекции в качестве входных данных для функции. Например, скажем, у вас есть массив `numbers`,
```julia
numbers = [4, 3, 2]
```
и вы хотите использовать `numbers` для создания произвольно заполненного массива $ 4 \times3 \times3 $ с помощью` rand`. `rand (numbers)` не будет делать то, что вы хотите. Вы можете индексировать в `numbers`, чтобы получить нужные значения и передать их в` rand`, как в
```julia
rand(numbers[1], numbers[2], numbers[3])
```
или вы можете использовать сплат:
```julia
rand(numbers...)
```
Используйте `...` для передачи содержимого `inputs`

```julia
inputs = [30, 12, "cats"]
```

в функцию `dreams`

```julia
dreams(i, j, perfect_mammal) = "I wish I had $(i + j) $perfect_mammal."
```

#### Упражнение 15:

Напишите функцию `partial`, чтобы вычислить $ i $-ю частную производную функции. Эта функция должна иметь четыре входа

* функция $ f $, для которой вы хотите вычислить частную производную
* массив *p*, задающий значения всех входных аргументов для $ f $ в той точке, где вы хотите вычислить $\frac{\partial f}{\partial p_i}$ 
* индекс переменной $ i $, по которой вы хотите вычислить частную производную $f$
* размер шага со значением по умолчанию 0.001

Подсказка: вам нужно `copy` и изменить` p` внутри `partial`.

## Градиентный спуск в 2-х измерениях

Оказывается, вектор градиента функции $ L_2 (w, b) $ задает направление на плоскости $ (w, b) $, в которой функция $ L_2 $ **увеличивается быстрее всего**. Другими словами, если вы начнете с позиции $ (w_0, b_0) $ и сделаете небольшой шаг длины $ \eta $ до новых значений $ (w ', b') $, значение функции изменится в новое значение $ L_2 (w ', b') $. Как максимально быстро добраться до минимума $ L_2 (w, b) $? 

Мы хотим сделать шаг в направлении, где $ L_2 $ уменьшается быстрее всего! В курсах по многопараметрическому исчислению показано, что $ L_2 $ будет *увеличиваться быстрее*, если вы сделаете шаг **в направлении градиента $ \nabla L_2 (w, b) $**! Чтобы максимально быстро уменьшить $ L_2 $, мы должны сделать шаг в *противоположном* направлении, $ - \nabla L_2 (w, b) $. Давайте теперь обобщим алгоритм градиентного спуска, который написали ранее, для работы с нашей двумерной функцией.

#### Упражнение 16

Расширьте вашу 1D-реализацию градиента, чтобы минимизировать функцию $L_2$.

Требования:

* Ваш новый метод для `gradient_descent` будет принимать четыре входных аргумента: функцию $f$ для которой вы ищете минимум, диапазон значений для первого входного аргумента $f$, который вы будете рассматривать, диапазон значений для второго аргумента $f$ и погрешность, которая будет указывать максимально допустимый размер шага, $\sum_i \eta \frac{\partial f}{\partial p_i}$

* Испольуйте $\eta = .01$. Например, для функции $f(w, b)$, меняйте $w$ как $w_{n+1} = w_n - 0.01 * \frac{\partial f}{\partial w_n}$

* Инициируйте `gradient_descent` с начальными координатами [-2.0, -2.0], то есть $w_0 = -2.0$ и $b_0 = -2.0$.

* Вернуть все шаги (их координаты), сделанные во время градиентного спуска, и значения функции потерь в этих координатах.

Как только будет готово, выполните

```julia
gradient_descent(L2, -2:0.02:2, -2:0.02:2, .001)
```

Сколько шагов было сделано при градиентном спуске? 

Подсказка: не считайте ваши начальные координаты `[-2.0, -2.0]` как шаг.

#### Упражнение 17

Используйте команды `surface` и` scatter! `, чтобы проиллюстрировать путь, пройденный` Gradient_descent` от [-2.0, -2.0] до того места, где заканчивается алгоритм.

Где рассеянные точки, представляющие ступени градиентного спуска, оказываются наиболее плотными?

A) Рядом с отправной точкой в [-2.0, -2.0]<br>
B) Рядом с [-1.8, 0] где $C_2$ кажется почти плоским<br>
C) Около минимума $C_2$

## Что мы узнали

Напомним, что в этом блокноте мы видели, что компьютер может вычислять приблизительные производные, и что мы можем использовать их в относительно простом алгоритме, градиентном спуске, чтобы минимизировать функции от 1 и 2 переменных! 

Теперь, когда мы использовали градиентный спуск, чтобы **узнать** наилучшие значения `b` и` w` для функции стоимости, мы можем представить, как мы могли бы сделать нашу классификацию яблок и бананов, позволив машине выбрать наши параметры! Для этого нам нужно определить функцию потерь для классификации яблок и бананов, используя изображения, которые уже были классифицированы как яблоки или бананы.